<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Video-Numbers" data-toc-modified-id="Video-Numbers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Video Numbers</a></span></li><li><span><a href="#Gender-Balance" data-toc-modified-id="Gender-Balance-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Gender Balance</a></span><ul class="toc-item"><li><span><a href="#Face-Size" data-toc-modified-id="Face-Size-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Face Size</a></span></li></ul></li><li><span><a href="#Donald-Trump-and-Hillary-Clinton" data-toc-modified-id="Donald-Trump-and-Hillary-Clinton-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Donald Trump and Hillary Clinton</a></span></li><li><span><a href="#James-Comey" data-toc-modified-id="James-Comey-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>James Comey</a></span></li><li><span><a href="#Shootings" data-toc-modified-id="Shootings-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Shootings</a></span></li></ul></div>

In [ ]:
from esper.stdlib import *
from esper.prelude import *
from esper.spark_util import *
from esper.validation import *
from esper.plot_util import plot_binary_proportion_comparison, plot_binary_screentime_proportion_comparison, \
    tile_images, plot_bar_chart, plot_time_series
from esper.spark_identity import get_screen_time_by_canonical_show_spark
from esper.identity_clusters import identity_clustering_workflow
from esper.major_canonical_shows import MAJOR_CANONICAL_SHOWS
from datetime import timedelta, datetime
from collections import OrderedDict
import random

# Video Numbers

In [ ]:
videos = get_videos()
print('Total seconds of 3 years data')
videos.where(videos.threeyears_dataset == True).select('duration').groupBy().sum().collect()

# Gender Balance

In [ ]:
faces = get_faces()
face_genders = get_face_genders()
# speakers = get_speakers()

In [ ]:
face_genders_nh = face_genders.where(face_genders.host_probability < 0.5)
screen_time_male, screen_time_female = OrderedDict(), OrderedDict()
screen_time_nh_male, screen_time_nh_female = OrderedDict(), OrderedDict()
screen_time_male['All Channels'] = sum_over_column(
    face_genders, 'duration', probability_column='male_probability'
)
screen_time_female['All Channels'] = sum_over_column(
    face_genders, 'duration', probability_column='female_probability'
)
screen_time_nh_male['All Channels'] = sum_over_column(
    face_genders_nh, 'duration', probability_column='male_probability'
)
screen_time_nh_female['All Channels'] = sum_over_column(
    face_genders_nh, 'duration', probability_column='female_probability'
)
channel_id_map = {c.id : c.name for c in Channel.objects.all()}
for k, v in sum_over_column(face_genders, 'duration', ['channel_id'], probability_column='male_probability').items():
    screen_time_male[channel_id_map[k[0]]] = v
for k, v in sum_over_column(face_genders, 'duration', ['channel_id'], probability_column='female_probability').items():
    screen_time_female[channel_id_map[k[0]]] = v
for k, v in sum_over_column(face_genders_nh, 'duration', ['channel_id'], probability_column='male_probability').items():
    screen_time_nh_male[channel_id_map[k[0]]] = v
for k, v in sum_over_column(face_genders_nh, 'duration', ['channel_id'], probability_column='female_probability').items():
    screen_time_nh_female[channel_id_map[k[0]]] = v

In [ ]:
# speaking_time_male, speaking_time_female = OrderedDict(), OrderedDict()

# gender_id_map = {g.id : g.name for g in Gender.objects.all()}
# for x in speakers.groupBy('gender_id').sum('duration').collect():
#     gender_id = x['gender_id']
#     value = x['sum(duration)'] 
#     if gender_id_map[gender_id] == 'M':
#         speaking_time_male['All Channels'] = (value, 0)
#     elif gender_id_map[gender_id] == 'F': 
#         speaking_time_female['All Channels'] = (value, 0)

# channel_id_map = {c.id : c.name for c in Channel.objects.all()}
# for x in speakers.groupBy('gender_id', 'channel_id').sum('duration').collect():
#     channel_id, gender_id = x['channel_id'], x['gender_id']
#     value = x['sum(duration)'] 
#     if gender_id_map[gender_id] == 'M':
#         speaking_time_male[channel_id_map[channel_id]] = (value, 0)
#     elif gender_id_map[gender_id] == 'F': 
#         speaking_time_female[channel_id_map[channel_id]] = (value, 0)

In [ ]:
sort_order = ['All Channels'] + [c.name for c in Channel.objects.all().order_by('name')]
plot_binary_proportion_comparison(
    ['Male', 'Female'], [screen_time_male, screen_time_female],
    '', '', 'Proportion of Screen Time',
    raw_data_to_label_fn=None, figsize=(5,5), sort_order=sort_order,
    legend_loc=4,
    save_path='figures/gender-3y-screen-time.pdf'
)
plot_binary_proportion_comparison(
    ['Male', 'Female'], [screen_time_nh_male, screen_time_nh_female],
    '', '', 'Proportion of Screen Time (Excluding Hosts)',
    raw_data_to_label_fn=None, figsize=(5,5), sort_order=sort_order,
    legend_loc=4,
    save_path='figures/gender-3y-screen-time-no-host.pdf'
)
# plot_binary_proportion_comparison(
#     ['Male', 'Female'], [speaking_time_male, speaking_time_female],
#     '', '', 'Proportion of Speaking Time',
#     raw_data_to_label_fn=None, figsize=(5,5), sort_order=sort_order,
#     legend_loc=4,
#     save_path='figures/gender-3y-speaking-time.pdf'
# )

In [ ]:
canonical_show_map = {c.id : c.name for c in CanonicalShow.objects.all() if c.name in MAJOR_CANONICAL_SHOWS}
screen_time_male_by_show, screen_time_female_by_show = {}, {}
for k, v in sum_over_column(face_genders, 'duration', ['canonical_show_id'],
                            probability_column='male_probability').items():
    if k[0] in canonical_show_map:
        screen_time_male_by_show[canonical_show_map[k[0]]] = v
for k, v in sum_over_column(face_genders, 'duration', ['canonical_show_id'], 
                            probability_column='female_probability').items():
    if k[0] in canonical_show_map:
        screen_time_female_by_show[canonical_show_map[k[0]]] = v

In [ ]:
plot_binary_proportion_comparison(
    ['Male', 'Female'], [screen_time_male_by_show, screen_time_female_by_show],
    'Gender Distribution of Screen Time vs. Show', '', 'Proportion of Screen Time',
    raw_data_to_label_fn=None, legend_loc=4,
    baseline_series_names=[
        'Baseline Male (Entire Dataset)', 
        'Baseline Female (Entire Dataset)'
    ],
    baseline_data=[
        screen_time_male['All Channels'][0],
        screen_time_female['All Channels'][0]
    ],
    save_path='figures/gender-3y-screen-time-by-show.pdf'
)

In [ ]:
show_order = [
    (k, screen_time_male_by_show[k][0]/screen_time_female_by_show[k][0]) 
    for k in screen_time_male_by_show
]
show_order.sort(key=lambda x: x[1])
canonical_show_rmap = { v : k for k, v in canonical_show_map.items()}
def face_img(face):
    return crop(load_frame(face.person.frame.video, face.person.frame.number, []), face)
def get_face_for_show(canonical_show_name, n_samples=10, dim=(100, 100)):
    print('Loading host faces:', canonical_show_name, 
          [x.name for x in CanonicalShow.objects.filter(name=canonical_show_name)[0].hosts.all()])
    host_faces = faces.where(
        (faces.canonical_show_id == canonical_show_rmap[canonical_show_name]) &
        (faces.host_probability >= 0.9)
    ).sample(0.1, False).select('id').limit(n_samples)
    host_face_ids = [h['id'] for h in host_faces.collect()]
    host_face_imgs = par_for(face_img, list(Face.objects.filter(id__in=host_face_ids)), progress=False)
    host_face_imgs = [cv2.resize(im, dim) for im in host_face_imgs]
    mean_host_im = np.zeros(host_face_imgs[0].shape)
    for im in host_face_imgs:
        mean_host_im += im
    mean_host_im /= len(host_face_imgs)
    mean_host_im = mean_host_im.astype(np.uint8)
    return mean_host_im
# show_imgs = [get_face_for_show(x[0]) for x in show_order]
show_imgs = par_for(get_face_for_show, [x[0] for x in show_order], workers=4)

In [ ]:
assert len(show_order) == len(show_imgs)
host_montage_img = np.hstack(show_imgs) 
plt.imsave('figures/show-host-avgs.jpg', cv2.cvtColor(host_montage_img, cv2.COLOR_RGB2BGR))
imshow(host_montage_img)
plt.show()

In [ ]:
screen_time_nh_male_by_show, screen_time_nh_female_by_show = {}, {}
for k, v in sum_over_column(face_genders_nh, 'duration', ['canonical_show_id'],
                            probability_column='male_probability').items():
    if k[0] in canonical_show_map:
        screen_time_nh_male_by_show[canonical_show_map[k[0]]] = v
for k, v in sum_over_column(face_genders_nh, 'duration', ['canonical_show_id'], 
                            probability_column='female_probability').items():
    if k[0] in canonical_show_map:
        screen_time_nh_female_by_show[canonical_show_map[k[0]]] = v

In [ ]:
plot_binary_proportion_comparison(
    ['Male (Excl. Hosts)', 'Female (Excl. Hosts)'], [screen_time_nh_male_by_show, screen_time_nh_female_by_show],
    'Gender Distribution of Screen Time (Excluding Hosts) vs. Show', '', 'Proportion of Screen Time',
    tertiary_series_names=['Male (Incl. Hosts)', 'Female (Incl. Hosts)'],
    tertiary_data=[screen_time_male_by_show, screen_time_female_by_show],
#     baseline_data=[
#         screen_time_male['All Channels'][0],
#         screen_time_female['All Channels'][0]
#     ],
#     baseline_series_names=[
#         'Baseline Male (Entire Dataset)', 
#         'Baseline Female (Entire Dataset)'
#     ],
    raw_data_to_label_fn=None, legend_loc=4,
    save_path='figures/gender-3y-screen-time-by-show-no-host.pdf'
)

In [ ]:
# speaking_time_male_by_show, speaking_time_female_by_show = {}, {}
# gender_id_map = {g.id : g.name for g in Gender.objects.all()}
# for x in speakers.groupBy('gender_id', 'canonical_show_id').sum('duration').collect():
#     canonical_show_id, gender_id = x['canonical_show_id'], x['gender_id']
#     value = x['sum(duration)'] 
#     if canonical_show_id in canonical_show_map:
#         if gender_id_map[gender_id] == 'M':
#             speaking_time_male_by_show[canonical_show_map[canonical_show_id]] = (value, 0)
#         elif gender_id_map[gender_id] == 'F': 
#             speaking_time_female_by_show[canonical_show_map[canonical_show_id]] = (value, 0)

In [ ]:
# plot_binary_proportion_comparison(
#     ['Male (Speaking Time)', 'Female (Speaking Time)'], [speaking_time_male_by_show, speaking_time_female_by_show],
#     'Gender Distribution of Speaking Time vs. Show', '', 
#     'Proportion of Speaking Time',
#     tertiary_series_names=['Male (Screen Time)', 'Female (Screen Time)'],
#     tertiary_data=[screen_time_male_by_show, screen_time_female_by_show],
#     raw_data_to_label_fn=None, legend_loc=4,
#     save_path='figures/gender-3y-speaking-time-by-show.pdf'
# )

In [ ]:
topic_to_male_screentime, topic_to_female_screentime = ({'abortion': (518294.8301809259, 298815.40602040046),
  'asthma': (339359.0225043069, 83964.431388244),
  'autism': (27679.673446264405, 13107.08559609269),
  'billy bush': (101162.10326908741, 32683.306271344332),
  'black lives': (422790.7060048257, 239574.11285279744),
  'border wall': (790434.1866875102, 349148.14352520194),
  'charleston church': (111839.93039077295, 59441.16777088708),
  'charlie hebdo': (664279.8498326603, 316250.8262504142),
  'collusion': (1883202.7915484093, 702020.2183103692),
  'email scandal': (489111.91803879297, 244568.81745981754),
  'facebook': (207257.28808802023, 66669.78404600148),
  'fashion': (12073.615677556301, 4569.36211837889),
  'flu': (981581.2873386245, 260143.71532067092),
  'gay marriage': (445052.66437707335, 222697.5446355079),
  'global warming': (176237.54147089276, 66528.90981610242),
  'harassment': (951201.1457089252, 398445.83589243225),
  'healthcare': (445370.4566424923, 215268.4113285826),
  'immigration': (956091.5229206171, 374548.43553251063),
  'irs': (332584.19550720387, 132787.31971463218),
  'isis': (2866859.4547577044, 1051335.1956239496),
  'lgbt': (148333.08406840145, 110610.45807212126),
  'nutrition': (628045.8830887104, 185545.78221704502),
  'paris attacks': (148227.55932958846, 52373.97777546179),
  'paris climate': (268291.2096124075, 102148.83588868032),
  'planned parenthood': (353526.37934005586, 204356.7247447404),
  'preschool': (153.00593562841277, 63.89683154229696),
  'public transportation': (1679.6862711204346, 409.727894334501),
  'pulse nightclub': (164350.70266352687, 79156.20542516006),
  'refugee': (626117.5072668382, 299180.83991466195),
  'roy moore': (1212925.1526060633, 499823.0925165218),
  'russia': (1564083.5109740235, 576207.0672263282),
  'san bernardino': (550743.7456960728, 301999.6689024732),
  'school': (1096258.8891856289, 802791.9080621441),
  'shooting': (1944700.2649337302, 1301799.5821853601),
  'syria': (2529187.136078459, 953596.1678198774),
  'terrorism': (315358.44025278196, 235902.7938372964),
  'travel': (276649.8170080849, 87448.70633821092),
  'travel ban': (1225879.3096513962, 511255.4354447183),
  'trayvon martin': (118247.72912627066, 47049.936429644426),
  'vacation': (130426.90833713741, 31574.75246438502),
  'vegas shooter': (179437.5703041849, 88900.48992088546),
  'walter scott': (1177897.8184054664, 701703.5025185598),
  'wedding': (96797.98378302893, 31697.39640199462),
  'yoga': (125075.1809051474, 48030.61142450444)},
 {'abortion': (394137.7796622501, 298815.40602040046),
  'asthma': (208911.96254740923, 83964.431388244),
  'autism': (14950.859419935132, 13107.08559609269),
  'billy bush': (55367.25108526693, 32683.306271344332),
  'black lives': (212890.4334346132, 239574.11285279744),
  'border wall': (325678.33149734064, 349148.14352520194),
  'charleston church': (61331.51772400852, 59441.16777088708),
  'charlie hebdo': (296120.4230068453, 316250.8262504142),
  'collusion': (680603.7657088493, 702020.2183103692),
  'email scandal': (320724.8044169953, 244568.81745981754),
  'facebook': (124714.53346686772, 66669.78404600148),
  'fashion': (8531.399337458719, 4569.36211837889),
  'flu': (594491.6604091169, 260143.71532067092),
  'gay marriage': (218912.6439312347, 222697.5446355079),
  'global warming': (72682.40917975792, 66528.90981610242),
  'harassment': (710866.3959327155, 398445.83589243225),
  'healthcare': (243335.22283698624, 215268.4113285826),
  'immigration': (448896.932901871, 374548.43553251063),
  'irs': (145823.0120336703, 132787.31971463218),
  'isis': (1195396.1967271124, 1051335.1956239496),
  'lgbt': (80428.90558792154, 110610.45807212126),
  'nutrition': (409477.65571675915, 185545.78221704502),
  'paris attacks': (61736.90880521267, 52373.97777546179),
  'paris climate': (109756.92522242744, 102148.83588868032),
  'planned parenthood': (282592.4308367877, 204356.7247447404),
  'preschool': (119.4682051790624, 63.89683154229696),
  'public transportation': (890.4835653827256, 409.727894334501),
  'pulse nightclub': (83570.31802382709, 79156.20542516006),
  'refugee': (287557.25836546073, 299180.83991466195),
  'roy moore': (698018.300413622, 499823.0925165218),
  'russia': (604499.5506876372, 576207.0672263282),
  'san bernardino': (250488.22734363307, 301999.6689024732),
  'school': (700394.4628330563, 802791.9080621441),
  'shooting': (895033.9286932275, 1301799.5821853601),
  'syria': (1023902.3067643837, 953596.1678198774),
  'terrorism': (112485.19605018759, 235902.7938372964),
  'travel': (154149.9636392286, 87448.70633821092),
  'travel ban': (583350.5388637857, 511255.4354447183),
  'trayvon martin': (60963.2658353575, 47049.936429644426),
  'vacation': (69082.22125912484, 31574.75246438502),
  'vegas shooter': (92242.2866527717, 88900.48992088546),
  'walter scott': (549474.6485948666, 701703.5025185598),
  'wedding': (54230.675876630434, 31697.39640199462),
  'yoga': (80957.50711620713, 48030.61142450444)})

In [ ]:
MIN_TOPIC_SECONDS = 60 * 60

def topic_order(a, b):
    keys = set()
    keys.update(a.keys())
    keys.update(b.keys())
    kv = []
    for key in keys:
        a_val = a.get(key, None)
        a_val = 1e-12 if a_val is None else a_val[0]
        b_val = b.get(key, None)
        b_val = 1e-12 if b_val is None else b_val[0]
        if a_val + b_val < MIN_TOPIC_SECONDS:
            continue
        kv.append((key, a_val / (a_val + b_val)))
    kv.sort(key=lambda x: x[1])
    return [k for k, _ in kv]

def sum_screentime(a, b, scale):
    result = defaultdict(int)
    for k, v in a.items():
        result[k] += v[0] / scale
    for k, v in b.items():
        result[k] += v[0] / scale
    return result


plot_binary_proportion_comparison(
    ['Male', 'Female'], 
    [topic_to_male_screentime, topic_to_female_screentime],
    'Gender Distribution of Screen Time By Topic',
    '', #'Topic',
    'Proportion of Screen Time',
    baseline_series_names=[
        'Baseline Male (All Topics)', 
        'Baseline Female (All Topics)'
    ],
    baseline_data=[
        screen_time_male['All Channels'][0],
        screen_time_female['All Channels'][0]
    ],
    sort_order=topic_order(topic_to_male_screentime, topic_to_female_screentime),
    raw_data_to_label_fn=None,
    legend_loc=4,
    save_path='figures/gender-3y-screen-time-by-topic.pdf'
)
plot_bar_chart(
    ['Male + Female'],
    [sum_screentime(topic_to_male_screentime, topic_to_female_screentime, 3600)],
    '',
    '', 'Coverage (Hours)',
    figsize=(14, 3), series_colors=['DarkGray'], logy=True, 
    show_ebars=False,
    sort_order=topic_order(topic_to_male_screentime, topic_to_female_screentime),
    save_path='figures/gender-3y-screen-time-by-topic-bar.pdf'
)

In [ ]:
topic_to_male_mentions, topic_to_female_mentions = ({'abortion': (11236.252132713795, 382.40713417580355),
  'affordable care act': (9022.325020432472, 178.28504708439905),
  'aids': (2994.206518113613, 80.62711396996598),
  'american health care act': (548.6910315752029, 9.816415056553373),
  'autism': (685.6003779172897, 19.605769053409603),
  'back to school': (445.91698437929153, 15.036586701108051),
  'birth control': (790.9575905799866, 32.12937705150955),
  'bombing': (10231.748476147652, 229.1596507367798),
  'breast cancer': (492.06045562028885, 22.139430128109463),
  'bullying': (2346.1988904476166, 67.76246679085271),
  'business': (91257.73589783907, 2072.886070510939),
  'cancer': (9490.374693930149, 318.74710125370245),
  'cervical cancer': (70.39318698644638, 2.297435426974729),
  'children': (47735.92115211487, 1609.6054159668056),
  'cholesterol': (246.24713438749313, 9.821888506982539),
  'christianity': (1735.81572920084, 39.62376633400151),
  'circumcision': (17.202438831329346, 0.8035193532600359),
  'congress': (121732.1553336978, 2253.507099359199),
  'contraceptive': (157.66393542289734, 5.82611281824118),
  'cooking': (1148.379305779934, 41.961463827020744),
  'cryptocurrency': (7.937322318553925, 0.06052572306270672),
  'diabetes': (1112.9122722148895, 58.72374729423551),
  'divorce': (2735.0869675278664, 109.48168286765573),
  'drugs': (13366.578389525414, 326.1088055093154),
  'erectile dysfunction': (254.83499819040298, 51.33387254796956),
  'evangelicals': (4828.962978422642, 108.15808383762996),
  'fashion': (7054.989916741848, 156.77042280390162),
  'feminism': (468.977885723114, 18.501466772520207),
  'harassment': (7634.083441615105, 189.8308387639827),
  'hiv': (734.9643259644508, 22.25738581115057),
  'hurricane': (15104.720743238926, 322.8347270487651),
  'immigration': (62325.57675892115, 1267.2134431873847),
  'influenza': (72.6507505774498, 1.772447656163245),
  'isis': (128508.38950520754, 2466.1025610262595),
  'islam': (17140.667168080807, 339.810690308258),
  'lgbt': (2946.194204747677, 82.94918957705578),
  'marijuana': (4614.809868454933, 119.50699899956558),
  'marriage': (15073.139418661594, 501.26221638148064),
  'massage': (393.5154851078987, 12.879459333881503),
  'medicine': (3812.5256739854813, 102.60356735899562),
  'millenial': (163.8210887312889, 3.597961624162618),
  'mom': (12879.210283339024, 549.0580769178864),
  'murder': (28123.603552222252, 803.5589044841867),
  'national security': (58577.55921572447, 1074.3868943708096),
  'nuclear': (60281.08161979914, 1208.832217647058),
  'nutrition': (419.5790290236473, 20.9405775310446),
  'obamacare': (58588.064242362976, 1065.369007634213),
  'obesity': (342.2370947599411, 6.054233961473762),
  'opiods': (30.247527480125427, 0.37525487239480065),
  'parenting': (403.7154778242111, 14.166476912983285),
  'pregnancy': (760.0743358135223, 26.53600939549398),
  'prostate cancer': (135.67578560113907, 3.3653219598482544),
  'prostitution': (444.2284245491028, 9.917093827082414),
  'racism': (10115.139577507973, 220.70818832073974),
  'sex education': (8.993772327899935, 0.3695955099834366),
  'sexism': (1769.061091542244, 45.874647775469356),
  'social security': (6086.034533202648, 122.3461074949366),
  'stem': (1172.9344388246536, 26.100557967605553),
  'teens': (1628.375369668007, 47.36954342383142),
  'terrorism': (41543.62926822901, 725.19879581253),
  'trumpcare': (937.4060715436935, 23.588299493523905),
  'violence': (39127.363469302654, 921.3077047006511),
  'weight loss': (105.67184317111969, 5.434820085540551),
  'women': (97541.35803723335, 3141.858557934175)},
 {'abortion': (8481.747867286205, 382.40713417580355),
  'affordable care act': (4258.674979567528, 178.28504708439905),
  'aids': (1712.7934818863869, 80.62711396996598),
  'american health care act': (197.30896842479706, 9.816415056553373),
  'autism': (507.39962208271027, 19.605769053409603),
  'back to school': (349.0830156207084, 15.036586701108051),
  'birth control': (867.0424094200134, 32.12937705150955),
  'bombing': (4642.251523852348, 229.1596507367798),
  'breast cancer': (694.9395443797112, 22.139430128109463),
  'bullying': (1922.8011095523834, 67.76246679085271),
  'business': (50323.26410216093, 2072.886070510939),
  'cancer': (6103.625306069851, 318.74710125370245),
  'cervical cancer': (64.60681301355362, 2.297435426974729),
  'children': (35583.07884788513, 1609.6054159668056),
  'cholesterol': (166.75286561250687, 9.821888506982539),
  'christianity': (744.18427079916, 39.62376633400151),
  'circumcision': (36.797561168670654, 0.8035193532600359),
  'congress': (53979.844666302204, 2253.507099359199),
  'contraceptive': (169.33606457710266, 5.82611281824118),
  'cooking': (913.6206942200661, 41.961463827020744),
  'cryptocurrency': (1.062677681446075, 0.06052572306270672),
  'diabetes': (824.0877277851105, 58.72374729423551),
  'divorce': (1853.9130324721336, 109.48168286765573),
  'drugs': (6005.4216104745865, 326.1088055093154),
  'erectile dysfunction': (700.165001809597, 51.33387254796956),
  'evangelicals': (2247.0370215773582, 108.15808383762996),
  'fashion': (3500.010083258152, 156.77042280390162),
  'feminism': (676.022114276886, 18.501466772520207),
  'harassment': (5656.916558384895, 189.8308387639827),
  'hiv': (550.0356740355492, 22.25738581115057),
  'hurricane': (8933.279256761074, 322.8347270487651),
  'immigration': (30598.423241078854, 1267.2134431873847),
  'influenza': (34.3492494225502, 1.772447656163245),
  'isis': (58204.61049479246, 2466.1025610262595),
  'islam': (6615.332831919193, 339.810690308258),
  'lgbt': (1935.8057952523232, 82.94918957705578),
  'marijuana': (2183.190131545067, 119.50699899956558),
  'marriage': (9189.860581338406, 501.26221638148064),
  'massage': (320.4845148921013, 12.879459333881503),
  'medicine': (2062.4743260145187, 102.60356735899562),
  'millenial': (157.1789112687111, 3.597961624162618),
  'mom': (12824.789716660976, 549.0580769178864),
  'murder': (14401.396447777748, 803.5589044841867),
  'national security': (26564.44078427553, 1074.3868943708096),
  'nuclear': (24202.918380200863, 1208.832217647058),
  'nutrition': (397.4209709763527, 20.9405775310446),
  'obamacare': (24388.935757637024, 1065.369007634213),
  'obesity': (228.7629052400589, 6.054233961473762),
  'opiods': (9.752472519874573, 0.37525487239480065),
  'parenting': (412.2845221757889, 14.166476912983285),
  'pregnancy': (826.9256641864777, 26.53600939549398),
  'prostate cancer': (49.32421439886093, 3.3653219598482544),
  'prostitution': (201.77157545089722, 9.917093827082414),
  'racism': (4984.860422492027, 220.70818832073974),
  'sex education': (36.00622767210007, 0.3695955099834366),
  'sexism': (1756.938908457756, 45.874647775469356),
  'social security': (2628.965466797352, 122.3461074949366),
  'stem': (680.0655611753464, 26.100557967605553),
  'teens': (1158.624630331993, 47.36954342383142),
  'terrorism': (16324.370731770992, 725.19879581253),
  'trumpcare': (376.59392845630646, 23.588299493523905),
  'violence': (20052.636530697346, 921.3077047006511),
  'weight loss': (178.3281568288803, 5.434820085540551),
  'women': (94146.64196276665, 3141.858557934175)})

In [ ]:
MIN_TOPIC_MENTIONS = 50

def topic_order(a, b):
    keys = set()
    keys.update(a.keys())
    keys.update(b.keys())
    kv = []
    for key in keys:
        a_val = a.get(key, None)
        a_val = 1e-12 if a_val is None else a_val[0]
        b_val = b.get(key, None)
        b_val = 1e-12 if b_val is None else b_val[0]
        if a_val + b_val < MIN_TOPIC_MENTIONS:
            continue
        kv.append((key, a_val / (a_val + b_val)))
    kv.sort(key=lambda x: x[1])
    return [k for k, _ in kv]

def sum_mentions(a, b):
    result = defaultdict(int)
    for k, v in a.items():
        result[k] += v[0]
    for k, v in b.items():
        result[k] += v[0]
    return result

plot_binary_proportion_comparison(
    ['Male', 'Female'], 
    [topic_to_male_mentions, topic_to_female_mentions],
    'On-Screen Gender Distribution By Keyword/Phrase',
    '', #'Keyword',
    'Proportion of Faces',
    legend_loc=4,
    raw_data_to_label_fn=None,
    baseline_series_names=[
        'Baseline Male (All Topics)', 
        'Baseline Female (All Topics)'
    ],
    baseline_data=[
        screen_time_male['All Channels'][0],
        screen_time_female['All Channels'][0]
    ],
    sort_order=topic_order(topic_to_male_mentions, topic_to_female_mentions),
    save_path='figures/gender-3y-mentions-by-topic.pdf'
)
plot_bar_chart(
    ['Male + Female'],
    [sum_mentions(topic_to_male_mentions, topic_to_female_mentions)],
    '',
    '', 'Faces (Count)',
    figsize=(14, 3), series_colors=['DarkGray'], logy=True, 
    show_ebars=False,
    sort_order=topic_order(topic_to_male_mentions, topic_to_female_mentions),
    save_path='figures/gender-3y-mentions-by-topic-bar.pdf'
)

## Face Size

In [ ]:
gender_map = { g.id : g.name for g in Gender.objects.all() }
def get_gender_face_sizes():
    # This touches spark
    result = OrderedDict([('M', {}), ('F', {})])
    for (gender_id, height), v in count_distinct_over_column(
                face_genders.where(face_genders.probability > 0.5),
                distinct_columns=[], #['face_id'],
                group_by_columns=['gender_id', 'height'],
                group_by_key_fn=lambda x: (x[0], math.ceil(x[1] * 100))
            ).items():
        result[gender_map[gender_id]][height] = v[0]
    return result
gender_face_size_hists = get_gender_face_sizes()

In [ ]:
def get_normalized_gender_face_sizes(hists):
    result = defaultdict(dict)
    for gender_name, hist in hists.items():
        denom = sum(hist.values())
        for k, v in hist.items():
            result[gender_name][k] = v / denom
    return result
norm_gender_face_size_hists = get_normalized_gender_face_sizes(gender_face_size_hists)
        
def get_cdf_gender_face_sizes(norm_hists):
    cdf = defaultdict(dict)
    for gender_name, hist in norm_hists.items():
        acc = 0.
        for k, v in sorted(hist.items(), key=lambda x: x[0]):
            cdf[gender_name][k] = acc
            acc += v
    return cdf
cdf_gender_face_size = get_cdf_gender_face_sizes(norm_gender_face_size_hists)

In [ ]:
def plot_curves(curves, title, x_label, y_label,
                color_map,
                y_lim=None, x_lim=None, figsize=(7, 4),
                save_path=None):
    fig, ax1 = plt.subplots(figsize=figsize)

    for series, values in curves.items():
        x = []
        y = []
        for k in sorted(values):
            x.append(k)
            y.append(values[k])
        ax1.plot(x, y, '-', label=series, 
                 color=color_map[series])
        
    ax1.legend()
    ax1.set_title(title)
    ax1.set_ylabel(y_label)
    ax1.set_xlabel(x_label)
    if y_lim is not None:
        ax1.set_ylim(y_lim)
    if x_lim is not None:
        ax1.set_xlim(x_lim)
    if save_path is None:
        plt.show()
    else:
        plt.tight_layout()
        plt.savefig(save_path)

def full_gender_name(d):
    result = {}
    gender_names = {'M': 'Male', 'F': 'Female'}
    for k, v in d.items():
        result[gender_names[k]] = v
    return result
        
gender_color_map = { 'Male': 'Blue', 'Female': 'Red' }
plot_curves(
    full_gender_name(norm_gender_face_size_hists), 
    'Distribution of Face Heights', 
    'Height (Percentage of Frame)', 'Proportion',
    gender_color_map,
    save_path='figures/gender-face-height-hist.pdf'
)
plot_curves(
    full_gender_name(cdf_gender_face_size), 
    'Cumulative Distribution of Face Heights', 
    'Height (Percentage of Frame)', 'CDF',
    gender_color_map,
    save_path='figures/gender-face-height-cdf.pdf'
)

# Donald Trump and Hillary Clinton 

In [ ]:
face_identities = get_face_identities()

In [ ]:
trump_clinton_names = ['donald trump', 'hillary clinton']

In [ ]:
channel_id_map = {c.id : c.name for c in Channel.objects.all()}

def person_helper(name, date_range):
    identity_id = Identity.objects.get(name=name).id
    person_identities = face_identities.where( 
        (face_identities.in_commercial == False) & 
        (face_identities.probability >= 0.3) &
        (face_identities.host_probability < 0.5) &
        (face_identities.identity_id == identity_id) &
        (face_identities.time >= func.to_date(func.lit(date_range[0]))) & 
        (face_identities.time < func.to_date(func.lit(date_range[1])))
    )
    screen_time_person = OrderedDict()
    screen_time_person['All Channels'] = sum_over_column(
        person_identities, 'duration', probability_column='probability'
    )
    for k, v in sum_over_column(person_identities, 'duration', ['channel_id'], 
                                probability_column='probability').items():
        screen_time_person[channel_id_map[k[0]]] = v
    return screen_time_person

trump_clinton_3mo = [person_helper(p, ['2016-08-01', '2016-11-09']) 
                     for p in trump_clinton_names]
trump_clinton_1yr = [person_helper(p, ['2015-11-09', '2016-11-09']) 
                     for p in trump_clinton_names]

In [ ]:
sort_order = ['All Channels'] + [c.name for c in Channel.objects.all().order_by('name')]
plot_binary_proportion_comparison(
    trump_clinton_names,
    trump_clinton_3mo,
    'Aug 1, 2016 to Nov 9, 2016 (3-Month)',
    '',
    'Proportion of Screen Time',
    series_colors=['LightSalmon', 'LightBlue'],
    error_colors=['Red', 'Blue'],
    legend_loc=4, figsize=(5, 5),
    raw_data_to_label_fn=None,
    sort_order=sort_order,
    save_path='figures/trump-clinton-3mo-ratio.pdf'
)
plot_binary_proportion_comparison(
    trump_clinton_names,
    trump_clinton_1yr,
    'Nov 9, 2015 to Nov 9, 2016 (1-Year)',
    '',
    'Proportion of Screen Time',
    series_colors=['LightSalmon', 'LightBlue'],
    error_colors=['Red', 'Blue'],
    legend_loc=4, figsize=(5, 5),
    raw_data_to_label_fn=None,
    sort_order=sort_order,
    save_path='figures/trump-clinton-1yr-ratio.pdf'
)

In [ ]:
trump_clinton_3mo_by_show = [
    get_screen_time_by_canonical_show_spark(
        name.lower(), 
        face_identities.where( 
            (face_identities.in_commercial == False) & 
            (face_identities.probability >= 0.3) &
            (face_identities.host_probability < 0.5)
        ),
        date_range=['2016-08-01', '2016-11-09']
    ) for name in trump_clinton_names
]
trump_clinton_1yr_by_show = [
    get_screen_time_by_canonical_show_spark(
        name.lower(), 
        face_identities.where( 
            (face_identities.in_commercial == False) & 
            (face_identities.probability >= 0.3) &
            (face_identities.host_probability < 0.5)
        ),
        date_range=['2015-11-09', '2016-11-09']
    ) for name in trump_clinton_names
]

In [ ]:
def convert_to_hours(d):
    return { k : (v[0].total_seconds() / 3600, v[1] / (3600 ** 2)) for k, v in d.items() }

def order_by_ratio(a, b):
    keys = set()
    keys.update(a.keys())
    keys.update(b.keys())
    kv = []
    for key in keys:
        a_val = a.get(key, None)
        a_val = 1e-12 if a_val is None else a_val[0].total_seconds()
        b_val = b.get(key, None)
        b_val = 1e-12 if b_val is None else b_val[0].total_seconds()
        kv.append((key, a_val / (a_val + b_val)))
    kv.sort(key=lambda x: x[1])
    return [k for k, _ in kv]

plot_bar_chart(
    trump_clinton_names, [convert_to_hours(x) for x in trump_clinton_3mo_by_show],
    'Screen Time Comparison from Aug 1, 2016 to Nov 9, 2016 (3-Month)',
    '', 'Hours', figsize=(14, 4), series_colors=['LightSalmon', 'LightBlue'],
    sort_order=order_by_ratio(*trump_clinton_3mo_by_show),
    save_path='figures/trump-clinton-by-show-3mo-bars.pdf'
)
plot_binary_proportion_comparison(
    trump_clinton_names,
    trump_clinton_3mo_by_show,
    'Distribution of Trump and Clinton Screen Time from Aug 1, 2016 to Nov 9, 2016 (3-Month)',
    '',
    'Proportion of Screen Time',
    series_colors=['LightSalmon', 'LightBlue'],
    error_colors=['Red', 'Blue'],
    legend_loc=4, figsize=(14, 5),
    raw_data_to_label_fn=None,
    sort_order=order_by_ratio(*trump_clinton_3mo_by_show),
    save_path='figures/trump-clinton-by-show-3mo-ratio.pdf'
)

plot_bar_chart(
    trump_clinton_names, [convert_to_hours(x) for x in trump_clinton_1yr_by_show],
    'Screen Time Comparison from Nov 9, 2015 to Nov 9, 2016 (1-Year)',
    '', 'Hours', figsize=(14, 4), series_colors=['LightSalmon', 'LightBlue'],
    sort_order=order_by_ratio(*trump_clinton_1yr_by_show),
    save_path='figures/trump-clinton-by-show-1yr-bars.pdf'
)
plot_binary_proportion_comparison(
    trump_clinton_names,
    trump_clinton_1yr_by_show,
    'Distribution of Trump and Clinton Screen Time from Nov 9, 2015 to Nov 9, 2016 (1-Year)',
    '',
    'Proportion of Screen Time',
    series_colors=['LightSalmon', 'LightBlue'],
    error_colors=['Red', 'Blue'],
    legend_loc=4, figsize=(14, 5),
    raw_data_to_label_fn=None,
    sort_order=order_by_ratio(*trump_clinton_1yr_by_show),
    save_path='figures/trump-clinton-by-show-1yr-ratio.pdf'
)

# James Comey

In [ ]:
comey_screen_time_by_show = get_screen_time_by_canonical_show_spark(
    'james comey', 
    face_identities.where( 
        (face_identities.in_commercial == False) & 
        (face_identities.probability >= 0.3) &
        (face_identities.host_probability < 0.5)
    ),
    date_range=['2016-01-01', '2018-01-01']
)

In [ ]:
plot_bar_chart(
    ['james comey'], [convert_to_hours(comey_screen_time_by_show)],
    'Screen Time from Jan 1, 2016 to Jan 1, 2018',
    '', 'Hours', figsize=(14, 6), series_colors=['LightSalmon'],
    save_path='figures/comey-by-show-1618.pdf'
)

In [ ]:
opinion_shows = [
    'The Rachel Maddow Show', 'Hardball',
    'Hannity', 'Tucker Carlson Tonight', 'The OReilly Factor',
]
opinion_show_channels = (['MSNBC'] * 2) + (['FOXNEWS'] * 3)
opinion_canonical_show_ids = { CanonicalShow.objects.get(name=s).id : s for s in opinion_shows } 

def comey_show_group_fn(x):
    canonical_show_id, timestring = x
    year, month, _ = timestring.split(' ')[0].split('-')
    return (canonical_show_id, datetime(year=int(year), month=int(month), day=15))
    

comey_date_range=['2016-01-01', '2018-01-01']
comey_opinion_screen_time_over_time = sum_over_column(
    face_identities.where(
        (face_identities.in_commercial == False) & 
        (face_identities.probability >= 0.3) &
        (face_identities.host_probability < 0.5) &
        (face_identities.time >= func.to_date(func.lit(comey_date_range[0]))) & 
        (face_identities.time < func.to_date(func.lit(comey_date_range[1]))) &
        (face_identities.canonical_show_id.isin(list(opinion_canonical_show_ids.keys())))
    ),
    'duration',
    ['canonical_show_id', 'time'],
    group_by_key_fn=comey_show_group_fn,
    probability_column='probability'
)

comey_opinion_screen_time_series = defaultdict(dict)
for (s, t), v in comey_opinion_screen_time_over_time.items():
    comey_opinion_screen_time_series[opinion_canonical_show_ids[s]][t] = v[0] / 3600 # hours

In [ ]:
channel_ids = { x.id : x.name for x in Channel.objects.all() }
channels = list(sorted(channel_ids.values()))

def comey_channel_group_fn(x):
    channel_id, timestring = x
    year, month, _ = timestring.split(' ')[0].split('-')
    return (channel_id, datetime(year=int(year), month=int(month), day=15))

comey_channel_screen_time_over_time = sum_over_column(
    face_identities.where(
        (face_identities.in_commercial == False) & 
        (face_identities.probability >= 0.3) &
        (face_identities.host_probability < 0.5) &
        (face_identities.time >= func.to_date(func.lit(comey_date_range[0]))) & 
        (face_identities.time < func.to_date(func.lit(comey_date_range[1])))
    ),
    'duration',
    ['channel_id', 'time'],
    group_by_key_fn=comey_channel_group_fn,
    probability_column='probability'
)

comey_channel_screen_time_series = defaultdict(dict)
for (c, t), v in comey_channel_screen_time_over_time.items():
    comey_channel_screen_time_series[channel_ids[c]][t] = v[0] / 3600 # hours

In [ ]:
plot_time_series(
    ['{} ({})'.format(opinion_shows[i], opinion_show_channels[i]) for i in range(len(opinion_shows))] ,
    [comey_opinion_screen_time_series[x] for x in opinion_shows], 
    'James Comey: Screen Time and Timeline', 
    'Hours',
#     max_time=datetime(year=2017, month=11, day=1),
    plotstyle='-', linewidth=2, figsize=(14, 4),
    discrete_events=[
        ('Recommends no charges against Clinton', (datetime(year=2016, month=7, day=5), 18.5)),
        ('Reopens Clinton email investigation', (datetime(year=2016, month=10, day=28), 5)),
        ('Trump wins presidential election', (datetime(year=2016, month=11, day=8), 3.5)),
        ('Informs Trump that he is not under investigation ', (datetime(year=2017, month=1, day=6), 17)),
        ('Testifies before House Intelligence Committee', (datetime(year=2017, month=3, day=20), 15.5)),
        ('Testifies before Senate Judiciary Committee', (datetime(year=2017, month=5, day=3), 14)),
        ('Fired by Trump', (datetime(year=2017, month=5, day=9), 12.5)),
        ('Mueller appointed as special counsel', (datetime(year=2017, month=5, day=17), 11)),
    ],
    save_path='figures/comey-opinion-time-series.pdf'
)

plot_time_series(
    channels,
    [comey_channel_screen_time_series[x] for x in channels], 
    'James Comey: Screen Time and Timeline', 
    'Hours',
#     max_time=datetime(year=2017, month=11, day=1),
    plotstyle='-', linewidth=2, figsize=(14, 4),
#     discrete_events=[
#         ('Recommends no charges against Clinton', (datetime(year=2016, month=7, day=5), 85)),
#         ('Reopens Clinton email investigation', (datetime(year=2016, month=10, day=28), 75)),
#         ('Trump wins presidential election', (datetime(year=2016, month=11, day=8), 65)),
#         ('Informs Trump that he is not under investigation ', (datetime(year=2017, month=1, day=6), 55)),
#         ('Testifies before House Intelligence Committee', (datetime(year=2017, month=3, day=20), 45)),
#         ('Testifies before Senate Judiciary Committee', (datetime(year=2017, month=5, day=3), 35)),
#         ('Fired by Trump', (datetime(year=2017, month=5, day=9),25)),
#         ('Mueller appointed as special counsel', (datetime(year=2017, month=5, day=17), 15)),
#     ],
    save_path='figures/comey-channel-time-series.pdf'
)

In [ ]:
opinion_host_to_shots = {
    'Sean Hannity': [
        584292, 584307, 6062987, 22151574, 27039495, 28005192, 28088277, 29310912, 37307674, 41194177, 
        44419012, 44591079, 44746669, 45164767, 47459051, 49952789, 59278225, 59278241, 60303840
    ], 
    'Bill O\'Reilly': [113168, 113621, 2526491, 10713245, 10714080],
    'Tucker Carlson': [157794, 21838447, 23198946, 44426599, 59737649, 52388910, 32578439, 21683169], 
    'Rachel Maddow': [1678041, 33194816, 44357358, 52691455], 
    'Chris Matthews': [
        5848914, 8589521, 11292838, 19482668, 22458768, 22458768, 32191124, 41072250, 45188880,
        48495239, 50722074, 54541370, 58423117, 58423117
    ]      
}

def montage_shots(shots, cols=4, limit=8):
    imgs = [(shot.id, load_frame(shot.video, int((shot.max_frame + shot.min_frame) / 2), []))
            for shot in Shot.objects.filter(id__in=shots).order_by('video__time')]
    if len(imgs) > limit: 
        imgs = [img for _, img in sorted(random.sample(list(enumerate(imgs)), limit))]
    print('selected:', [i for i, _ in imgs])
    shape = (imgs[0][1].shape[1], imgs[0][1].shape[0])
    imgs = [cv2.resize(img, shape) for _, img in imgs]
    return tile_images(imgs, cols=cols, blank_value=255)

for name, shots in opinion_host_to_shots.items():
    print(name)
    last_name = name.split(' ')[-1].replace("'", '')
    img_out_path = 'figures/comey-{}.jpg'.format(last_name.lower())
    img_montage = montage_shots(shots)
    plt.imsave(img_out_path, cv2.cvtColor(img_montage, cv2.COLOR_RGB2BGR))
    imshow(img_montage)
    plt.show()

In [ ]:
for year in [2015, 2016, 2017]:
    print(year)
    for name, shots in opinion_host_to_shots.items():
        last_name = name.split(' ')[-1].replace("'", '')
        img_out_path = 'figures/comey-{}-{}.jpg'.format(last_name.lower(), year)
        shots_in_year = [x.id for x in Shot.objects.filter(id__in=shots, video__time__year=year)]
        if len(shots_in_year) > 0:
            print(name)
            img_montage = montage_shots(shots_in_year)
#             plt.imsave(img_out_path, cv2.cvtColor(img_montage, cv2.COLOR_RGB2BGR))
            imshow(img_montage)
            plt.show()

# Shootings

In [ ]:
raise Exception('This section requires human effort!')

In [ ]:
EXAMPLES_PER_CLUSTER = 5

In [ ]:
identity_clustering_workflow(
    'Christian Taylor', 
    examples_per_cluster=EXAMPLES_PER_CLUSTER, 
    init_clusters=20, 
    merge_cluster_threshold=0.05,
    show_titles=False,
    face_probability_threshold=0.5,
    duration_label_unit='s',
    save_paths=[
        'figures/victim-taylor-screen-time.pdf', 
        'figures/victim-taylor-row-norm.pdf', 
        'figures/victim-taylor-col-norm.pdf'
    ]
)

In [ ]:
identity_clustering_workflow(
    'Syed Rizwan Farook', 
    examples_per_cluster=EXAMPLES_PER_CLUSTER, 
    init_clusters=25, 
    merge_cluster_threshold=0.05,
    show_titles=False,
    face_probability_threshold=0.5,
    save_paths=[
        'figures/shooter-farook-screen-time.pdf', 
        'figures/shooter-farook-row-norm.pdf', 
        'figures/shooter-farook-col-norm.pdf'
    ]
)

In [ ]:
identity_clustering_workflow(
    'Tashfeen Malik', 
    examples_per_cluster=EXAMPLES_PER_CLUSTER, 
    init_clusters=25,
    merge_cluster_threshold=0.05,
    show_titles=False,
    face_probability_threshold=0.5,
    save_paths=[
        'figures/shooter-malik-screen-time.pdf', 
        'figures/shooter-malik-row-norm.pdf', 
        'figures/shooter-malik-col-norm.pdf'
    ]
)